In [1]:
import os 

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Udemy\\ML_Bootcamp\\18_End_to_End_DataScience_Project\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

##### Define ConfigurationManager

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_dictionaries

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_dictionaries([self.config.artifacts_root])
        
        
    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        
        create_dictionaries([config.root_dir])
        
        model_trainer_config= ModelTrainingConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )
        
        return model_trainer_config
        

In [6]:
import pandas as pd
import os 
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

In [15]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        
        
        train_x=train_data.drop([self.config.target_column],axis=1)
        train_y=train_data[self.config.target_column]
        
        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[self.config.target_column]
        
        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        

In [16]:
try:
    config= ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    logger.info(f"Model training completed successfully.")
except Exception as e:
    logger.error(f"Error occurred during model training: {str(e)}")
    raise e

[2025-02-18 07:00:35,382] INFO: common: yaml file: <_io.TextIOWrapper name='D:\\Udemy\\ML_Bootcamp\\18_End_to_End_DataScience_Project\\datascienceproject\\src\\datascience\\config\\config.yaml' mode='r' encoding='UTF-8'> loaded successfully:]
[2025-02-18 07:00:35,384] INFO: common: yaml file: <_io.TextIOWrapper name='D:\\Udemy\\ML_Bootcamp\\18_End_to_End_DataScience_Project\\datascienceproject\\params.yaml' mode='r' encoding='UTF-8'> loaded successfully:]
[2025-02-18 07:00:35,386] INFO: common: yaml file: <_io.TextIOWrapper name='D:\\Udemy\\ML_Bootcamp\\18_End_to_End_DataScience_Project\\datascienceproject\\schema.yaml' mode='r' encoding='UTF-8'> loaded successfully:]
[2025-02-18 07:00:35,387] INFO: common: Created directory at: artifacts.:]
[2025-02-18 07:00:35,388] INFO: common: Created directory at: artifacts/model_trainer.:]
[2025-02-18 07:00:35,400] INFO: 2756337894: Model training completed successfully.:]
